In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D

In [ ]:
# Load data
df = pd.read_csv("space_mountain_with_holiday_weather_lag_suite.csv")

# Parse date
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
df.sort_values('Datetime', inplace=True)

# One-hot encode Day of Week
df = pd.get_dummies(df, columns=['Day of Week'], drop_first=True)

# Drop unused columns
df.drop(columns=['Date', 'Time', 'Datetime'], inplace=True)

# Scale all features
scaler = StandardScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)

In [ ]:
SEQUENCE_LENGTH = 12
X, y = [], []
features = scaled_df.drop(columns=['Wait Time']).values
wait_values = scaled_df['Wait Time'].values

for i in range(SEQUENCE_LENGTH, len(scaled_df)):
    X.append(features[i-SEQUENCE_LENGTH:i])
    y.append(wait_values[i])

X = np.array(X)
y = np.array(y)

# Train/test split
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
# Predict
y_pred = model.predict(X_test)

# Inverse transform to real wait time
wait_scaler = StandardScaler()
wait_scaler.fit(df[['Wait Time']])
y_pred_unscaled = wait_scaler.inverse_transform(y_pred)
y_test_unscaled = wait_scaler.inverse_transform(y_test.reshape(-1, 1))

real_rmse = np.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
print(f"Conv1D Unscaled RMSE (in minutes): {real_rmse:.2f}")